# Torch Sequential 실습

In [1]:
import torch
from torch import nn, optim
from torch.nn import functional as F

### Torch Tensors

 - gpu 사용이 가능한지 확인한 후 사용 가능하면 'cuda' 아니면 'cpu' 값을 device에 설정하세요.

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

- 리스트 [1,2,3,4]를 tensor로 변환하세요.

In [3]:
t = torch.tensor([1,2,3,4])
t

tensor([1, 2, 3, 4])

- 리스트 [1,2,3,4]를 float type tensor로 변환하세요.

In [4]:
t = torch.Tensor([1,2,3,4])
t

tensor([1., 2., 3., 4.])

- 리스트 [1,2,3,4]를 float type tensor로 변환하세요.

In [5]:
t = torch.FloatTensor([1,2,3,4])
t

tensor([1., 2., 3., 4.])

- 리스트 [1,2,3,4]를 float64 type tensor로 변환하세요.

In [6]:
t = torch.DoubleTensor([1,2,3,4])
t

tensor([1., 2., 3., 4.], dtype=torch.float64)

- 리스트 [1,2,3,4]를 정수 타입 tensor로 변환하세요.

In [7]:
t = torch.IntTensor([1,2,3,4])
t

tensor([1, 2, 3, 4], dtype=torch.int32)

- 리스트 [1,2,3,4]를 정수64 타입 tensor로 변환하세요.

In [8]:
t = torch.LongTensor([1,2,3,4])
t

tensor([1, 2, 3, 4])

- t의 차원 크기를 조회하세요.

In [9]:
t.dim()

1

- t의 shape을 조회하세요.

In [10]:
t.shape

torch.Size([4])

In [11]:
t.size()

torch.Size([4])

- t의 shape을 [1,4]로 변경하세요.

In [11]:
t.reshape(1, -1)

tensor([[1, 2, 3, 4]])

In [12]:
t.view(1, -1)

tensor([[1, 2, 3, 4]])

- t에 축을 하나 추가하고 t에 저장하세요.

In [13]:
t = t.unsqueeze(axis=1)
t

tensor([[1],
        [2],
        [3],
        [4]])

- t의 shape을 조회하세요.

In [15]:
t.shape

torch.Size([4, 1])

- t의 새 축을 0번 축에 추가하고 저장하세요.

In [16]:
t = t.unsqueeze(axis=0)
t

tensor([[[1],
         [2],
         [3],
         [4]]])

- t의 shape을 조회하세요.

In [17]:
t.shape

torch.Size([1, 4, 1])

- t의 맨 마지막 축을 제거하고 t에 저장하세요.

In [18]:
t = t.squeeze(axis=2)
t

tensor([[1, 2, 3, 4]])

- t의 shape을 조회하세요.

In [21]:
t.shape

torch.Size([1, 4])

- t를 복사하여 t2에 저장하세요.

In [23]:
t2 = t.clone().detach()
t2

tensor([[1, 2, 3, 4]])

- t2를 numpy array로 변환하세요.

In [24]:
t2.detach().numpy()

array([[1, 2, 3, 4]], dtype=int64)

- t를 device에 할당한 후 t에 저장하세요.

In [25]:
t = t.to(device)
t

tensor([[1, 2, 3, 4]], device='cuda:0')

- t를 numpy array로 변환하세요.

In [26]:
t.cpu().numpy()

array([[1, 2, 3, 4]], dtype=int64)

- t를 numpy array로 변환하세요.

In [27]:
t.detach().cpu().numpy()

array([[1, 2, 3, 4]], dtype=int64)

### Torch Sequential Model

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

 - 다이아몬드 가격을 Sequential 모델로 예측

- diamonds.csv 로드

In [29]:
diamond = pd.read_csv('./data/diamonds.csv')
diamond = diamond.drop(diamond.columns[0], axis=1)

- 문자열 인코딩

In [30]:
def encode_cut(x):
    return {'Fair':0, 'Good':1, 'Very Good':2, 'Premium':3, 'Ideal':4}[x]

def encode_color(x):
    return abs(ord(x)-74)

def encode_clarity(x):
    return {'I1':0, 'SI2':1, 'SI1':2, 'VS2':3, 'VS1':4, 'VVS2':5, 'VVS1':6, 'IF':7}[x]

diamond['cut'] = diamond['cut'].apply(encode_cut)
diamond['color'] = diamond['color'].apply(encode_color)
diamond['clarity'] = diamond['clarity'].apply(encode_clarity)

- 데이터 선택

In [31]:
features = [
    'carat',
    'cut',
    'color',
    'clarity',
    'depth',
    'table',
    'x',
    'y',
    'z'
]
X = diamond[features]
Y = diamond['price']

- 데이터 분할

In [32]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y,
                                                    test_size=.3,
                                                    random_state=1)

In [33]:
x_train.shape

(37758, 9)

- 비교를 위해 GrandientBoosting으로 모델링한 결과를 출력하세요.
- 평가 측도는 mse, $r^2$입니다.

In [34]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

model = GradientBoostingRegressor(random_state=0)
model.fit(x_train, y_train)
mean_squared_error(y_train, model.predict(x_train)), mean_squared_error(y_test, model.predict(x_test))

(363227.58354604786, 386611.2288453066)

In [35]:
model.score(x_train, y_train), model.score(x_test, y_test)

(0.9775249015488457, 0.9747968787640697)

- 시드를 777로 설정하세요.

In [76]:
torch.manual_seed(777)

- Sequential 모델을 이용하여 DNN을 구성하세요.
- 두 개의 은닉층은 각각 64, 32개 퍼셉트론
- 활성화는 relu

In [77]:
model  = nn.Sequential(
    nn.Linear(x_train.shape[1], 64),
    nn.ReLU(),
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 1),
).to(device)

- 모델 구조를 출력하세요.

In [78]:
print(model)

Sequential(
  (0): Linear(in_features=9, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=32, bias=True)
  (3): ReLU()
  (4): Linear(in_features=32, out_features=1, bias=True)
)


- 모델의 summary를 출력하세요.

In [79]:
from torchinfo import summary

summary(model)

Layer (type:depth-idx)                   Param #
Sequential                               --
├─Linear: 1-1                            640
├─ReLU: 1-2                              --
├─Linear: 1-3                            2,080
├─ReLU: 1-4                              --
├─Linear: 1-5                            33
Total params: 2,753
Trainable params: 2,753
Non-trainable params: 0

- 데이터에 적합한 loss와 optimizer를 선정하여 설정하세요.

In [80]:
loss = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.1)

- train 데이터를 학습시키세요.
- epoch=10000회

In [81]:
from tqdm import notebook
n_epochs = 10000

X = torch.Tensor(x_train.values).to(device)
Y = torch.Tensor(y_train.values).to(device)


progressbar = notebook.tqdm(range(n_epochs))

for epoch in progressbar:
    h = model(X)
    cost = loss(h, Y.view(-1, 1))
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    progressbar.set_postfix({'Loss':f'{cost.item():12.6f}'})

  0%|          | 0/10000 [00:00<?, ?it/s]

- 학습시킨 모델로 mse, $r^2$를 출력하세요.

In [82]:
train_pred = model(torch.FloatTensor(x_train.values).to(device)).detach().cpu().numpy()
test_pred = model(torch.FloatTensor(x_test.values).to(device)).detach().cpu().numpy()

mean_squared_error(y_train, train_pred), mean_squared_error(y_test, test_pred)

(389169.41404462315, 388581.7863644072)

In [83]:
from sklearn.metrics import r2_score

r2_score(y_train, train_pred)

0.9759197228100325

- 시드를 777로 설정하세요.

In [84]:
torch.manual_seed(777)

- model의 구성을 변경하여 다시 학습해보세요.

In [85]:
model  = nn.Sequential(
    nn.Linear(x_train.shape[1], 256),
    nn.ReLU(),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 1),
).to(device)

- 모델의 summary를 출력하세요.

In [86]:
from torchinfo import summary

summary(model)

Layer (type:depth-idx)                   Param #
Sequential                               --
├─Linear: 1-1                            2,560
├─ReLU: 1-2                              --
├─Linear: 1-3                            32,896
├─ReLU: 1-4                              --
├─Linear: 1-5                            8,256
├─ReLU: 1-6                              --
├─Linear: 1-7                            2,080
├─ReLU: 1-8                              --
├─Linear: 1-9                            33
Total params: 45,825
Trainable params: 45,825
Non-trainable params: 0

- 데이터에 적합한 loss와 optimizer를 선정하여 설정하세요.

In [87]:
loss = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)

- train 데이터를 학습시키세요.
- epoch=10000회

In [88]:
from tqdm import notebook
n_epochs = 10000

X = torch.Tensor(x_train.values).to(device)
Y = torch.Tensor(y_train.values).to(device)

progressbar = notebook.tqdm(range(n_epochs))

for epoch in progressbar:
    h = model(X)
    cost = loss(h, Y.view(-1, 1))
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    progressbar.set_postfix({'Loss':f'{cost.item():12.6f}'})

  0%|          | 0/10000 [00:00<?, ?it/s]

- 학습시킨 모델로 mse, $r^2$를 출력하세요.

In [89]:
train_pred = model(torch.FloatTensor(x_train.values).to(device)).detach().cpu().numpy()
test_pred = model(torch.FloatTensor(x_test.values).to(device)).detach().cpu().numpy()

mean_squared_error(y_train, train_pred), mean_squared_error(y_test, test_pred)

(280409.37142085715, 300536.31264251104)

In [90]:
r2_score(y_train, train_pred), r2_score(y_test, test_pred)

(0.9826493677385846, 0.9804080881304165)

 - 타이타닉 생존자를 Sequential 모델로 예측

- titanic_r.csv 파일 로드

In [93]:
df = pd.read_csv('./data/titanic_r.csv')
df

,survived,sex,age,sibsp,parch,fare,class,who,deck,embark_town
0,0,male,22.0,1,0,7.2500,Third,man,NaN,Southampton
1,1,female,38.0,1,0,71.2833,First,woman,C,Cherbourg
2,1,female,26.0,0,0,7.9250,Third,woman,NaN,Southampton
3,1,female,35.0,1,0,53.1000,First,woman,C,Southampton
4,0,male,35.0,0,0,8.0500,Third,man,NaN,Southampton
...,...,...,...,...,...,...,...,...,...,...
886,0,male,27.0,0,0,13.0000,Second,man,NaN,Southampton
887,1,female,19.0,0,0,30.0000,First,woman,B,Southampton
888,0,female,NaN,1,2,23.4500,Third,woman,NaN,Southampton
889,1,male,26.0,0,0,30.0000,First,man,C,Cherbourg


- 데이터 전처리
  - 새 열 child: who 값이 child면 1 아니면 0
  - 새 열 family: sibsp + parch
  - 새 열 alone: family 값이 0보다 크면 0 아니면 0

In [94]:
df['child'] = df['who'].apply(lambda x: 1 if x=='child' else 0)
df['family'] = df['sibsp'] + df['parch']
df['alone'] = df['family'].apply(lambda x: 0 if x>0 else 1)

- 데이터 인코딩
  - class 열: First 1, Second 2, Third 3
  - deck 열: A->0, B->1 순으로 순서대로 부여, 결측치는 그대로 결측치
  - embark_town 열: Queenstown 0, Cherbourg 1, Southampton 2 결측치는 그대로 결측치
  - sex 열: female 0, male 1

In [95]:
def encode_class(x):
    return {'First':1, 'Second':2, 'Third':3}[x]

def encode_deck(x):
    if type(x) == str:
        return ord(x)-ord('A')
    else:
        return x

def encode_town(x):
    if type(x) == str:
        return {'Queenstown':0, 'Cherbourg':1, 'Southampton':2}[x]
    else:
        return x

df['class'] = df['class'].apply(encode_class)
df['deck'] = df['deck'].apply(encode_deck)
df['embark_town'] = df['embark_town'].apply(encode_town)
df['sex'] = df['sex'].apply(lambda x: 0 if x=='female' else 1)

- who 열 삭제

In [96]:
df = df.drop('who', axis=1)
df

,survived,sex,age,sibsp,parch,fare,class,deck,embark_town,child,family,alone
0,0,1,22.0,1,0,7.2500,3,NaN,2.0,0,1,0
1,1,0,38.0,1,0,71.2833,1,2.0,1.0,0,1,0
2,1,0,26.0,0,0,7.9250,3,NaN,2.0,0,0,1
3,1,0,35.0,1,0,53.1000,1,2.0,2.0,0,1,0
4,0,1,35.0,0,0,8.0500,3,NaN,2.0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,1,27.0,0,0,13.0000,2,NaN,2.0,0,0,1
887,1,0,19.0,0,0,30.0000,1,1.0,2.0,0,0,1
888,0,0,NaN,1,2,23.4500,3,NaN,2.0,0,3,0
889,1,1,26.0,0,0,30.0000,1,2.0,1.0,0,0,1


 - 결측치 처리
   - age 열: class, alone 그룹의 평균 나이
   - deck 열: -1
   - embark_town: embark_town의 최빈값

In [97]:
df['age'] = df['age'].fillna(df.groupby(['class', 'alone'])['age'].transform('mean'))
df['deck'] = df['deck'].fillna(-1)
df['embark_town'] = df['embark_town'].fillna(df['embark_town'].mode()[0])

 - 데이터 전처리2
   - 새 열 unk_deck = deck의 값이 -1이면 1 아니면 0

In [98]:
df['unk_deck'] = df['deck'].apply(lambda x: 1 if x==-1 else 0)
df

,survived,sex,age,sibsp,parch,fare,class,deck,embark_town,child,family,alone,unk_deck
0,0,1,22.000,1,0,7.2500,3,-1.0,2.0,0,1,0,1
1,1,0,38.000,1,0,71.2833,1,2.0,1.0,0,1,0,0
2,1,0,26.000,0,0,7.9250,3,-1.0,2.0,0,0,1,1
3,1,0,35.000,1,0,53.1000,1,2.0,2.0,0,1,0,0
4,0,1,35.000,0,0,8.0500,3,-1.0,2.0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,1,27.000,0,0,13.0000,2,-1.0,2.0,0,0,1,1
887,1,0,19.000,0,0,30.0000,1,1.0,2.0,0,0,1,0
888,0,0,19.784,1,2,23.4500,3,-1.0,2.0,0,3,0,1
889,1,1,26.000,0,0,30.0000,1,2.0,1.0,0,0,1,0


- 데이터 선택

In [99]:
features = [
    'sex',
    'age',
    'sibsp',
    'parch',
    'fare',
    'class',
    'deck',
    'embark_town',
    'child',
    'family',
    'alone',
    'unk_deck'
]
X = df[features]
Y = df['survived']

- 데이터 분할
  - 분할 비율 7:3
  - 층화 추출

In [100]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y,
                                                    test_size=.3,
                                                    random_state=0,
                                                    stratify=Y)

- 비교를 위해 GrandientBoosting으로 모델링한 결과를 출력하세요.
- 평가 측도는 accuracy입니다.

In [101]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

model = GradientBoostingClassifier(random_state=0)
model.fit(x_train, y_train)
accuracy_score(y_train, model.predict(x_train)), accuracy_score(y_test, model.predict(x_test))

(0.9069020866773676, 0.8470149253731343)

- 시드를 777로 설정하세요.

In [102]:
torch.manual_seed(777)

- Sequential 모델을 이용하여 DNN을 구성하세요.
- 두 개의 은닉층은 각각 64, 32개 퍼셉트론
- 은닉 활성화는 relu, 출력 활성화는 sigmoid

In [103]:
model  = nn.Sequential(
    nn.Linear(x_train.shape[1], 64),
    nn.ReLU(),
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 1),
    nn.Sigmoid()
).to(device)

 - 모델 구성 출력

In [104]:
print(model)

Sequential(
  (0): Linear(in_features=12, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=32, bias=True)
  (3): ReLU()
  (4): Linear(in_features=32, out_features=1, bias=True)
  (5): Sigmoid()
)


- 데이터에 적합한 loss와 optimizer를 선정하여 설정하세요.

In [105]:
loss = nn.BCELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [106]:
from tqdm import notebook
n_epochs = 10000

X = torch.Tensor(x_train.values).to(device)
Y = torch.Tensor(y_train.values).to(device)

progressbar = notebook.tqdm(range(n_epochs))

for epoch in progressbar:
    h = model(X)
    cost = loss(h, Y.view(-1, 1))
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    progressbar.set_postfix({'Loss':f'{cost.item():12.6f}'})

  0%|          | 0/10000 [00:00<?, ?it/s]

- 학습시킨 모델로 accuracy를 출력하세요.

In [107]:
train_pred = model(torch.FloatTensor(x_train.values).to(device)).detach().cpu().numpy()
test_pred = model(torch.FloatTensor(x_test.values).to(device)).detach().cpu().numpy()

train_pred= np.where(train_pred>0.5, 1, 0)
test_pred= np.where(test_pred>0.5, 1, 0)

accuracy_score(y_train, train_pred), accuracy_score(y_test, test_pred)

(0.9470304975922953, 0.7611940298507462)

- 반복 횟수를 줄여서 train 데이터를 학습시키세요.
- epoch=3000회

In [ ]:
torch.manual_seed(777)

In [116]:
model  = nn.Sequential(
    nn.Linear(x_train.shape[1], 64),
    nn.ReLU(),
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 1),
    nn.Sigmoid()
).to(device)

In [117]:
loss = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [118]:
from tqdm import notebook
n_epochs = 3000

X = torch.Tensor(x_train.values).to(device)
Y = torch.Tensor(y_train.values).to(device)

progressbar = notebook.tqdm(range(n_epochs))

for epoch in progressbar:
    h = model(X)
    cost = loss(h, Y.view(-1, 1))
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    progressbar.set_postfix({'Loss':f'{cost.item():12.6f}'})

  0%|          | 0/3000 [00:00<?, ?it/s]

- 학습시킨 모델로 accuracy를 출력하세요.

In [119]:
train_pred = model(torch.FloatTensor(x_train.values).to(device)).detach().cpu().numpy()
test_pred = model(torch.FloatTensor(x_test.values).to(device)).detach().cpu().numpy()

train_pred= np.where(train_pred>0.5, 1, 0)
test_pred= np.where(test_pred>0.5, 1, 0)

accuracy_score(y_train, train_pred), accuracy_score(y_test, test_pred)

(0.85553772070626, 0.8171641791044776)